# 📊 GCAA Facebook 社群分析 Dashboard

此 Notebook 從 Google Sheets 讀取 Facebook Insights 資料，產生可互動的視覺化圖表。

## 📑 Dashboard 結構

| 頁面 | 內容 | 用途 |
|------|------|------|
| **1. 總覽 Dashboard** | KPI 指標卡、週度趨勢圖、表現等級分布 | 快速掌握整體粉專表現 |
| **2. 時段分析** | 每小時互動率長條圖、最佳時段表格 | 找出最佳發文時間 |
| **3. 內容分析** | 行動類型表現、議題表現、交叉分析 | 分析不同內容策略成效 |
| **4. 投廣建議** | 投廣推薦清單、潛力貼文 | 廣告投放決策依據 |

---

## 📖 圖表閱讀指南

### 🎯 如何讀懂各類圖表

**KPI 指標卡**
- 總貼文數：一段時間內發布的貼文總量
- 平均互動率 (ER)：(互動總數 / 觸及人數) × 100%，業界基準約 1-3%
- 總觸及：所有貼文觸及的獨立用戶總數

**週度趨勢圖**
- 橫軸為時間（週），縱軸為指標數值
- 觀察趨勢走勢，尋找季節性或事件影響

**象限分析**
- 王牌貼文 (高觸及+高互動)：持續產出的標竿
- 潛力珍寶 (低觸及+高互動)：適合廣告放大
- 廣傳陷阱 (高觸及+低互動)：檢視內容品質
- 常態內容 (低觸及+低互動)：調整策略

---

## 🔧 Step 1: 環境設定與資料讀取

In [ ]:
# 安裝與導入必要套件
!pip install --quiet pandas gspread plotly google-auth-oauthlib kaleido

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import auth
import gspread
from google.auth import default
from IPython.display import display, HTML, Markdown

# Google Colab 授權
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ Google Colab 授權成功")

In [ ]:
# === 讀取所有資料源 ===
SHEET_URL = "https://docs.google.com/spreadsheets/d/17d4pELhu5ylTPHsG3mmbSDu1Fh9rzDWugqFZYJ1XAg0"

# 工作表與 GID 對照 (更新為實際 GID)
SHEETS_CONFIG = {
    'weekly_trends': None,
    'hourly_performance': None,
    'format_type_performance': None,
    'issue_topic_performance': None,
    'top_posts': None,
    'ad_recommendations': None,
    'raw_posts': None,
    'best_posting_times': None,
    'quadrant_analysis': None
}

def load_sheet(sheet_name):
    """從 Google Sheets 讀取指定工作表"""
    try:
        spreadsheet = gc.open_by_url(SHEET_URL)
        worksheet = spreadsheet.worksheet(sheet_name)
        data = worksheet.get_all_records()
        df = pd.DataFrame(data)
        print(f"  ✓ {sheet_name}: {len(df)} 筆")
        return df
    except Exception as e:
        print(f"  ✗ {sheet_name}: {e}")
        return pd.DataFrame()

print("📥 載入資料中...")
data = {}
for sheet_name in SHEETS_CONFIG.keys():
    data[sheet_name] = load_sheet(sheet_name)

print("\n✅ 資料載入完成")

---

## 📈 Page 1: 總覽 Dashboard

快速掌握粉專整體表現：KPI 指標、趨勢變化、表現分布。

### 📖 閱讀指南
- **KPI 卡片**：一目瞭然的關鍵指標
- **週度趨勢**：觀察長期走勢，識別高峰/低谷週
- **圓餅圖**：了解貼文表現分布，調整內容策略

In [ ]:
# === KPI 指標卡 ===
df_trends = data['weekly_trends']
df_posts = data['raw_posts']

if not df_trends.empty:
    # 計算 KPI
    total_posts = len(df_posts) if not df_posts.empty else df_trends['貼文數'].sum()
    avg_er = df_trends['平均互動率 (%)'].mean() if '平均互動率 (%)' in df_trends.columns else 0
    total_reach = df_trends['總觸及'].sum() if '總觸及' in df_trends.columns else 0
    total_engagement = df_trends['總互動數'].sum() if '總互動數' in df_trends.columns else 0

    # 建立 KPI 卡片
    fig_kpi = go.Figure()

    fig_kpi.add_trace(go.Indicator(
        mode="number",
        value=total_posts,
        title={"text": "<b>總貼文數</b>", "font": {"size": 16}},
        domain={'x': [0, 0.25], 'y': [0, 1]},
        number={'font': {'size': 48, 'color': '#1877F2'}}
    ))

    fig_kpi.add_trace(go.Indicator(
        mode="number",
        value=avg_er,
        title={"text": "<b>平均互動率 (%)</b>", "font": {"size": 16}},
        domain={'x': [0.25, 0.5], 'y': [0, 1]},
        number={'font': {'size': 48, 'color': '#42B72A'}, 'suffix': '%'}
    ))

    fig_kpi.add_trace(go.Indicator(
        mode="number",
        value=total_reach,
        title={"text": "<b>總觸及人數</b>", "font": {"size": 16}},
        domain={'x': [0.5, 0.75], 'y': [0, 1]},
        number={'font': {'size': 48, 'color': '#F7931A'}, 'valueformat': ',.0f'}
    ))

    fig_kpi.add_trace(go.Indicator(
        mode="number",
        value=total_engagement,
        title={"text": "<b>總互動數</b>", "font": {"size": 16}},
        domain={'x': [0.75, 1], 'y': [0, 1]},
        number={'font': {'size': 48, 'color': '#E0245E'}, 'valueformat': ',.0f'}
    ))

    fig_kpi.update_layout(
        title={'text': '📊 KPI 總覽', 'x': 0.5, 'font': {'size': 24}},
        height=200,
        margin=dict(t=60, b=20, l=20, r=20)
    )
    fig_kpi.show()
else:
    print("⚠️ 無法載入 weekly_trends 資料")

In [ ]:
# === 週度趨勢折線圖 ===
if not df_trends.empty and '週次 (日期範圍)' in df_trends.columns:
    fig_trend = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=['平均互動率 (%)', '總觸及人數'],
        vertical_spacing=0.12
    )

    # 互動率趨勢
    fig_trend.add_trace(
        go.Scatter(
            x=df_trends['週次 (日期範圍)'],
            y=df_trends['平均互動率 (%)'],
            mode='lines+markers',
            name='互動率',
            line=dict(color='#42B72A', width=2),
            marker=dict(size=6)
        ),
        row=1, col=1
    )

    # 觸及趨勢
    fig_trend.add_trace(
        go.Bar(
            x=df_trends['週次 (日期範圍)'],
            y=df_trends['總觸及'],
            name='觸及',
            marker_color='#1877F2',
            opacity=0.7
        ),
        row=2, col=1
    )

    fig_trend.update_layout(
        title={'text': '📈 週度趨勢分析', 'x': 0.5, 'font': {'size': 20}},
        height=600,
        showlegend=False,
        hovermode='x unified'
    )
    fig_trend.update_xaxes(tickangle=45)
    fig_trend.show()

    print("""\n📖 趨勢圖解讀:
    - 尋找趨勢的高峰週，分析當週發布了什麼類型的內容
    - 觀察互動率與觸及是否同步變化，或有分離現象
    - 若觸及高但互動低，可能是內容吸引力不足""")

In [ ]:
# === 表現等級分布圓餅圖 ===
df_top = data['top_posts']

if not df_top.empty and '表現等級' in df_top.columns:
    tier_counts = df_top['表現等級'].value_counts()

    colors = {
        '熱門 (前5%)': '#E0245E',
        '優質 (前25%)': '#42B72A',
        '一般 (中間50%)': '#1877F2',
        '待改進 (後25%)': '#657786'
    }

    fig_pie = go.Figure(data=[go.Pie(
        labels=tier_counts.index,
        values=tier_counts.values,
        marker_colors=[colors.get(t, '#999') for t in tier_counts.index],
        textinfo='label+percent',
        hovertemplate='<b>%{label}</b><br>數量: %{value}<br>佔比: %{percent}<extra></extra>'
    )])

    fig_pie.update_layout(
        title={'text': '🏆 貼文表現等級分布', 'x': 0.5, 'font': {'size': 20}},
        height=400
    )
    fig_pie.show()

    print("""\n📖 等級定義:
    - 熱門 (前5%): 互動率遠超同期貼文，值得複製成功模式
    - 優質 (前25%): 表現良好的貼文
    - 一般 (中間50%): 符合粉專平均水平
    - 待改進 (後25%): 可分析原因並調整策略""")

---

## ⏰ Page 2: 時段分析

找出粉絲最活躍的時間，優化發文排程。

### 📖 閱讀指南
- **每小時互動率**：找出一天中最佳發文時段
- **最佳時段表格**：結合星期與時段的最佳組合

In [ ]:
# === 每小時互動率長條圖 ===
df_hourly = data['hourly_performance']

if not df_hourly.empty and '時間' in df_hourly.columns:
    fig_hourly = go.Figure()

    fig_hourly.add_trace(go.Bar(
        x=df_hourly['時間'],
        y=df_hourly['平均互動率 (%)'],
        marker_color='#1877F2',
        text=df_hourly['平均互動率 (%)'].round(2),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>平均互動率: %{y:.2f}%<br>貼文數: %{customdata}<extra></extra>',
        customdata=df_hourly['貼文數'] if '貼文數' in df_hourly.columns else None
    ))

    # 標記最佳時段
    best_hour_idx = df_hourly['平均互動率 (%)'].idxmax()
    best_hour = df_hourly.loc[best_hour_idx, '時間']
    best_er = df_hourly.loc[best_hour_idx, '平均互動率 (%)']

    fig_hourly.add_annotation(
        x=best_hour,
        y=best_er,
        text=f"🏆 最佳時段",
        showarrow=True,
        arrowhead=2,
        arrowcolor='#E0245E',
        font=dict(color='#E0245E', size=12)
    )

    fig_hourly.update_layout(
        title={'text': '⏰ 每小時平均互動率', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='時間',
        yaxis_title='平均互動率 (%)',
        height=450,
        showlegend=False
    )
    fig_hourly.show()

    print(f"""\n📖 時段分析解讀:
    - 🏆 最佳發文時段: {best_hour} (平均互動率 {best_er:.2f}%)
    - 建議在互動率高峰前 30 分鐘發文，讓貼文有時間曝光
    - 同時參考貼文數量，樣本越大結論越可靠""")

In [ ]:
# === 最佳發文時段表格 ===
df_best = data['best_posting_times']

if not df_best.empty:
    # 篩選有效資料 (排除標題列)
    df_best_clean = df_best[df_best['時段'].notna() & (df_best['時段'] != '')].copy()

    if not df_best_clean.empty:
        # 建立熱力圖
        pivot_data = df_best_clean.pivot_table(
            index='時段',
            columns='星期',
            values='平均互動率 (%)',
            aggfunc='mean'
        )

        if not pivot_data.empty:
            fig_heatmap = go.Figure(data=go.Heatmap(
                z=pivot_data.values,
                x=pivot_data.columns,
                y=pivot_data.index,
                colorscale='RdYlGn',
                text=np.round(pivot_data.values, 2),
                texttemplate='%{text}%',
                hovertemplate='<b>%{y}</b> %{x}<br>互動率: %{z:.2f}%<extra></extra>'
            ))

            fig_heatmap.update_layout(
                title={'text': '📅 時段 × 星期 互動率熱力圖', 'x': 0.5, 'font': {'size': 20}},
                height=400,
                xaxis_title='星期',
                yaxis_title='時段'
            )
            fig_heatmap.show()

    print("""\n📖 熱力圖解讀:
    - 顏色越綠代表互動率越高
    - 找出 2-3 個最佳時段組合，作為固定發文排程
    - 綠色格子少的時段可視情況避開""")

---

## 📝 Page 3: 內容分析

分析不同類型的內容策略成效，找出最有效的組合。

### 📖 閱讀指南
- **行動類型**：不同發文形式（記者會、聲明稿、活動等）的效果
- **議題表現**：不同議題主題的受眾反應
- **交叉分析**：找出黃金組合

In [ ]:
# === 行動類型表現橫條圖 ===
df_format = data['format_type_performance']

if not df_format.empty and '行動' in df_format.columns:
    df_format_sorted = df_format.sort_values('平均互動率 (%)', ascending=True)

    fig_format = go.Figure()

    fig_format.add_trace(go.Bar(
        y=df_format_sorted['行動'],
        x=df_format_sorted['平均互動率 (%)'],
        orientation='h',
        marker_color='#1877F2',
        text=df_format_sorted['平均互動率 (%)'].round(2).astype(str) + '%',
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>平均互動率: %{x:.2f}%<br>貼文數: %{customdata}<extra></extra>',
        customdata=df_format_sorted['貼文數']
    ))

    fig_format.update_layout(
        title={'text': '🎯 行動類型表現比較', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='平均互動率 (%)',
        yaxis_title='',
        height=max(400, len(df_format_sorted) * 40),
        showlegend=False,
        margin=dict(l=150)
    )
    fig_format.show()

    # 找出最佳行動
    best_format = df_format.loc[df_format['平均互動率 (%)'].idxmax()]
    print(f"""\n📖 行動類型分析:
    - 🏆 最佳行動類型: {best_format['行動']} (ER: {best_format['平均互動率 (%)']:.2f}%)
    - 建議多產出高互動率類型的內容
    - 同時考慮貼文數量，樣本太少的結論有限""")

In [ ]:
# === 議題表現橫條圖 ===
df_issue = data['issue_topic_performance']

if not df_issue.empty and '議題' in df_issue.columns:
    df_issue_sorted = df_issue.sort_values('平均互動率 (%)', ascending=True)

    fig_issue = go.Figure()

    fig_issue.add_trace(go.Bar(
        y=df_issue_sorted['議題'],
        x=df_issue_sorted['平均互動率 (%)'],
        orientation='h',
        marker_color='#42B72A',
        text=df_issue_sorted['平均互動率 (%)'].round(2).astype(str) + '%',
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>平均互動率: %{x:.2f}%<br>貼文數: %{customdata}<extra></extra>',
        customdata=df_issue_sorted['貼文數']
    ))

    fig_issue.update_layout(
        title={'text': '📌 議題表現比較', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='平均互動率 (%)',
        yaxis_title='',
        height=max(350, len(df_issue_sorted) * 50),
        showlegend=False,
        margin=dict(l=150)
    )
    fig_issue.show()

    # 找出最佳議題
    best_issue = df_issue.loc[df_issue['平均互動率 (%)'].idxmax()]
    print(f"""\n📖 議題分析:
    - 🏆 最受歡迎議題: {best_issue['議題']} (ER: {best_issue['平均互動率 (%)']:.2f}%)
    - 受眾對不同議題的反應差異可能與時事、季節性有關
    - 可搭配時段分析，找出特定議題的最佳發文時機""")

In [ ]:
# === 象限分析散點圖 ===
df_quadrant = data['quadrant_analysis']

if not df_quadrant.empty and '觸及人數' in df_quadrant.columns and '互動率 (%)' in df_quadrant.columns:
    # 確保數據類型正確
    df_quadrant['觸及人數'] = pd.to_numeric(df_quadrant['觸及人數'], errors='coerce')
    df_quadrant['互動率 (%)'] = pd.to_numeric(df_quadrant['互動率 (%)'], errors='coerce')
    df_quadrant = df_quadrant.dropna(subset=['觸及人數', '互動率 (%)'])

    # 計算中位數
    median_reach = df_quadrant['觸及人數'].median()
    median_er = df_quadrant['互動率 (%)'].median()

    # 定義象限顏色
    quadrant_colors = {
        '王牌貼文': '#E0245E',
        '潛力珍寶': '#42B72A',
        '廣傳陷阱': '#F7931A',
        '常態內容': '#657786'
    }

    fig_quad = px.scatter(
        df_quadrant,
        x='觸及人數',
        y='互動率 (%)',
        color='象限' if '象限' in df_quadrant.columns else None,
        color_discrete_map=quadrant_colors,
        hover_data=['議題', '行動', '內容預覽'] if all(c in df_quadrant.columns for c in ['議題', '行動', '內容預覽']) else None,
        custom_data=['連結'] if '連結' in df_quadrant.columns else None
    )

    # 加入中位數線
    fig_quad.add_vline(x=median_reach, line_dash="dash", line_color="grey", opacity=0.5)
    fig_quad.add_hline(y=median_er, line_dash="dash", line_color="grey", opacity=0.5)

    # 加入象限標籤
    x_range = df_quadrant['觸及人數'].max() - df_quadrant['觸及人數'].min()
    y_range = df_quadrant['互動率 (%)'].max() - df_quadrant['互動率 (%)'].min()

    annotations = [
        dict(x=median_reach + x_range*0.3, y=median_er + y_range*0.3, text="👑 王牌貼文", showarrow=False, font=dict(size=14, color='#E0245E')),
        dict(x=median_reach - x_range*0.3, y=median_er + y_range*0.3, text="💎 潛力珍寶", showarrow=False, font=dict(size=14, color='#42B72A')),
        dict(x=median_reach + x_range*0.3, y=median_er - y_range*0.3, text="⚠️ 廣傳陷阱", showarrow=False, font=dict(size=14, color='#F7931A')),
        dict(x=median_reach - x_range*0.3, y=median_er - y_range*0.3, text="📝 常態內容", showarrow=False, font=dict(size=14, color='#657786'))
    ]

    fig_quad.update_layout(
        title={'text': '📊 貼文象限分析 (觸及 vs 互動)', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='觸及人數 (擴散力)',
        yaxis_title='互動率 % (吸引力)',
        height=600,
        annotations=annotations,
        legend_title='象限'
    )
    fig_quad.show()

    print(f"""\n📖 象限分析解讀:
    - 中位數觸及: {median_reach:,.0f} | 中位數互動率: {median_er:.2f}%

    👑 王牌貼文 (右上): 高觸及+高互動，是成功標竿
    💎 潛力珍寶 (左上): 低觸及+高互動，「最適合投廣」的貼文
    ⚠️ 廣傳陷阱 (右下): 高觸及+低互動，可能是標題黨或內容不符預期
    📝 常態內容 (左下): 可參考王牌貼文的特點來調整""")

    # 統計各象限數量
    if '象限' in df_quadrant.columns:
        quad_counts = df_quadrant['象限'].value_counts()
        print("\n各象限貼文數量:")
        for q, count in quad_counts.items():
            print(f"  {q}: {count} 篇 ({count/len(df_quadrant)*100:.1f}%)")

---

## 💰 Page 4: 投廣建議

根據貼文表現，提供廣告投放建議。

### 📖 閱讀指南
- **投廣推薦清單**：高潛力貼文的投放建議
- **潛力分數**：結合多項指標的綜合評估
- 點擊貼文連結可直接開啟 Facebook

In [ ]:
# === 投廣推薦清單 ===
df_ads = data['ad_recommendations']

if not df_ads.empty:
    # 顯示欄位
    display_cols = ['發布時間', '投廣建議', '潛力分數', '表現等級', '行動', '議題']
    available_cols = [c for c in display_cols if c in df_ads.columns]

    if available_cols:
        # 建立可點擊的表格
        df_display = df_ads[available_cols].head(20).copy()

        # 如果有潛力分數，按分數排序
        if '潛力分數' in df_display.columns:
            df_display = df_display.sort_values('潛力分數', ascending=False)

        print("📋 投廣推薦清單 (Top 20):\n")
        display(df_display.style.background_gradient(subset=['潛力分數'] if '潛力分數' in df_display.columns else [], cmap='RdYlGn'))

        print("""\n📖 投廣建議解讀:
        - 🎯 優先投廣: 高潛力分數 + 「潛力珍寶」象限的貼文
        - 💡 潛力分數: 綜合互動率、分享率、內容熱度的評估
        - ⏰ 建議投廣時機: 貼文發布後 24-48 小時內效果最佳
        - 💰 預算分配: 根據潛力分數比例分配，高分貼文多投""")

        # 投廣建議分布
        if '投廣建議' in df_ads.columns:
            rec_counts = df_ads['投廣建議'].value_counts()
            print("\n投廣建議分布:")
            for rec, count in rec_counts.head(5).items():
                print(f"  {rec}: {count} 篇")

In [ ]:
# === 潛力貼文視覺化 ===
if not df_ads.empty and '潛力分數' in df_ads.columns:
    df_ads['潛力分數'] = pd.to_numeric(df_ads['潛力分數'], errors='coerce')
    df_top_potential = df_ads.nlargest(15, '潛力分數')

    fig_potential = go.Figure()

    # 建立標籤
    labels = []
    for _, row in df_top_potential.iterrows():
        label = f"{row.get('行動', '')} | {row.get('議題', '')}"
        if '發布時間' in row:
            label = str(row['發布時間'])[:10] + " | " + label
        labels.append(label[:40])

    fig_potential.add_trace(go.Bar(
        y=labels,
        x=df_top_potential['潛力分數'],
        orientation='h',
        marker_color='#42B72A',
        text=df_top_potential['潛力分數'].round(1),
        textposition='outside'
    ))

    fig_potential.update_layout(
        title={'text': '💎 Top 15 高潛力投廣貼文', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='潛力分數',
        yaxis_title='',
        height=500,
        showlegend=False,
        margin=dict(l=200)
    )
    fig_potential.show()

---

## 📋 Dashboard 摘要

執行以下 Cell 產生一頁式摘要報告。

In [ ]:
# === 產生摘要報告 ===
print("="*60)
print("📊 GCAA Facebook 社群分析摘要")
print("="*60)

# KPI 摘要
if not data['weekly_trends'].empty:
    df_t = data['weekly_trends']
    print(f"\n📈 整體表現:")
    print(f"   總貼文數: {len(data['raw_posts']) if not data['raw_posts'].empty else 'N/A'}")
    print(f"   平均互動率: {df_t['平均互動率 (%)'].mean():.2f}%" if '平均互動率 (%)' in df_t.columns else "")
    print(f"   總觸及: {df_t['總觸及'].sum():,.0f}" if '總觸及' in df_t.columns else "")

# 最佳時段
if not data['hourly_performance'].empty:
    df_h = data['hourly_performance']
    if '平均互動率 (%)' in df_h.columns:
        best_h = df_h.loc[df_h['平均互動率 (%)'].idxmax()]
        print(f"\n⏰ 最佳發文時段: {best_h['時間']} (ER: {best_h['平均互動率 (%)']:.2f}%)")

# 最佳內容類型
if not data['format_type_performance'].empty:
    df_f = data['format_type_performance']
    if '平均互動率 (%)' in df_f.columns:
        best_f = df_f.loc[df_f['平均互動率 (%)'].idxmax()]
        print(f"\n🎯 最佳行動類型: {best_f['行動']} (ER: {best_f['平均互動率 (%)']:.2f}%)")

if not data['issue_topic_performance'].empty:
    df_i = data['issue_topic_performance']
    if '平均互動率 (%)' in df_i.columns:
        best_i = df_i.loc[df_i['平均互動率 (%)'].idxmax()]
        print(f"\n📌 最受歡迎議題: {best_i['議題']} (ER: {best_i['平均互動率 (%)']:.2f}%)")

# 投廣建議數量
if not data['ad_recommendations'].empty:
    df_a = data['ad_recommendations']
    print(f"\n💰 投廣建議: 共 {len(df_a)} 篇潛力貼文")

print("\n" + "="*60)
print("✅ 報告產生完成")